In [1]:
from layers.autoencoder.vae import AudioAutoencoder,OobleckDecoder,OobleckEncoder,AutoEncoderWrapper
import os
import torch



#model_state = torch.load("../models/Nov29.pth")
model_state = torch.load("../models/last.pth",map_location='cpu')
model = AudioAutoencoder(sample_rate=16000,downsampling_ratio=2048).to("cpu")
model.load_state_dict(model_state)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal Parameters: {total_params:,}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


wrapper = AutoEncoderWrapper(autoencoder_state_path="../models/final3.pth").to(device)

/var/folders/3q/svznhqyd5q73pvhf_d5zds9w0000gn/T/ipykernel_29320/3857798552.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state = torch.load("../models/last.pth"


Total Parameters: 159,088,798
Loaded Pretrained autoencoder. (CPU)


/Users/ssongjinseobgmail.com/coding/Re-Beethoven/src/layers/autoencoder/vae.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.ae.load_state_dict(torch.load(autoenco

In [5]:
import torchaudio
import torchaudio.transforms as T

def load_audio(file_path, target_sample_rate=16000):
    """
    Loads an audio file and resamples it to the target sample rate.

    Args:
        file_path (str): Path to the audio file.
        target_sample_rate (int): Desired sample rate. Defaults to 16kHz.

    Returns:
        torch.Tensor: Audio waveform as a 1D tensor.
    """
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)
    
    # If stereo, convert to mono by averaging the channels
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0)
    
    # Resample if necessary
    if sample_rate != target_sample_rate:
        resample = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resample(waveform)
    
    return waveform
import torchaudio

def save_audio(file_path, waveform, sample_rate=16000):
    """
    Saves a waveform as an audio file.

    Args:
        file_path (str): Path to save the audio file (e.g., "output.mp3").
        waveform (torch.Tensor): Audio waveform as a 1D or 2D tensor.
        sample_rate (int): Sample rate of the audio. Defaults to 16kHz.

    Returns:
        None
    """
    # Ensure waveform is 2D (channel, time), even if mono
    if waveform.dim() == 1:
        waveform = waveform.unsqueeze(0)
    
    # Save the audio
    torchaudio.save(file_path, waveform, sample_rate)

import torch
from layers.tools.losses import MultiResolutionSTFTLoss


# Main processing
input_path = "../dataset/no14/0/audio0.mp3"

    
waveform = load_audio(input_path, 16000).unsqueeze(0).to(device)  # 1D waveform
waveform = waveform[:, 0:16384*30].unsqueeze(0)
print(waveform.shape)
out = wrapper.encode_audio(waveform,overlap=0)
decode = wrapper.decode_audio(out,overlap=0)
decode.shape


torch.Size([1, 1, 491520])


torch.Size([1, 491520])

In [6]:

save_audio("out.wav", decode.cpu())

